# Velib

## Config

In [1]:
API_KEY = "a44bff4661b1435a625ea7b661a3f6410386bcf3"
BOOTSTRAP_SERVERS = 'localhost:9092'
# For several servers
#BOOTSTRAP_SERVERS = ['localhost:9092', 'localhost:9093']

## Install package `kafka-python`

In [2]:
import sys
!conda install --yes --prefix {sys.prefix} kafka-python

# Or via pip
#import sys
#!{sys.executable} -m pip install kafka-python

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - kafka-python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gsl-2.4                    |blas_openblash5c7cb9b_1004         2.5 MB  conda-forge
    libgcc-ng-7.3.0            |       hdf63c60_0         6.1 MB  conda-forge
    gfortran_linux-64-7.3.0    |       h553295d_3           9 KB  conda-forge
    pycurl-7.43.0.2            |   py36h1ba5d50_0         185 KB  defaults
    xorg-libsm-1.2.2           |       h470a237_5          24 KB  conda-forge
    kafka-python-1.4.4         |             py_0         166 KB  conda-forge
    gxx_impl_linux-64-7.3.0    |       hdf63c60_1        18.7 MB  conda-forge
    

## Kafka Producer (velib-get-stations.py)

In [7]:
import json
import time
import urllib.request

from kafka import KafkaProducer

#API_KEY = "XXX" # FIXME Set your own API key here
API_KEY = "a44bff4661b1435a625ea7b661a3f6410386bcf3"
url = "https://api.jcdecaux.com/vls/v1/stations?apiKey={}".format(API_KEY)
producer = KafkaProducer(bootstrap_servers='kafka:9092')

while True:
    response = urllib.request.urlopen(url)
    stations = json.loads(response.read().decode())
    for station in stations:
        producer.send("velib-stations", json.dumps(station).encode(),
                key=str(station["number"]).encode()) # dispatch each stations always on same partition
    print("{} Produced {} station records".format(time.time(), len(stations)))
    time.sleep(1)

1548861679.790327 Produced 2550 station records
1548861682.3572083 Produced 2550 station records
1548861685.1278825 Produced 2550 station records
1548861687.9457362 Produced 2550 station records
1548861690.5463557 Produced 2550 station records
1548861693.1583445 Produced 2550 station records
1548861695.9666407 Produced 2550 station records
1548861698.5614603 Produced 2550 station records
1548861701.4568229 Produced 2550 station records
1548861704.0445135 Produced 2550 station records
1548861706.7118747 Produced 2550 station records
1548861709.3782177 Produced 2550 station records
1548861712.1387393 Produced 2550 station records
1548861714.850036 Produced 2550 station records
1548861717.5447726 Produced 2550 station records
1548861720.0994985 Produced 2550 station records
1548861722.6562173 Produced 2550 station records
1548861725.419466 Produced 2550 station records
1548861727.9179971 Produced 2550 station records
1548861730.8366015 Produced 2550 station records
1548861733.5093045 Prod

KeyboardInterrupt: 

## Kafka Broker

In [ ]:
%%bash
./bin/zookeeper-server-start.sh ./config/zookeeper.properties
./bin/kafka-server-start.sh ./config/server.properties
./bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic velib-stations

## Kafka Consumer (velib-monitor-stations.py)

In [ ]:
import json
from kafka import KafkaConsumer

stations = {}
consumer = KafkaConsumer("velib-stations", bootstrap_servers=BOOTSTRAP_SERVERS, group_id="velib-monitor-stations")

for message in consumer:
    station = json.loads(message.value.decode())
    station_number = station["number"]
    contract = station["contract_name"]
    available_bike_stands = station["available_bike_stands"]

    if contract not in stations:
        stations[contract] = {}

    city_stations = stations[contract]

    if station_number not in city_stations:
        city_stations[station_number] = available_bike_stands

    count_diff = available_bike_stands - city_stations[station_number]

    if count_diff != 0:
        city_stations[station_number] = available_bike_stands
        print("{}{} {} ({})".format(
            "+" if count_diff > 0 else "",
            count_diff, station["address"], contract
        ))

## Modification de la durée de conservation des messages dans Kafka

Suppression d'un segment toutes les 4 secondes

In [ ]:
! ./bin/kafka-configs.sh --zookeeper localhost:2181 --entity-type topics --entity-name velib-stations --alter --add-config retention.ms=4000

Modification de la longueur d'un segment

In [ ]:
! ./bin/kafka-configs.sh --zookeeper localhost:2181 --entity-type topics --entity-name velib-stations --alter --add-config segment.ms=2000